# Темы семинара: отбор признаков и приложения

- Фильтрационные методы
- Оберточные методы
- Встроенные методы
- Нелинейные классификаторы
- Сохранение модели

In [1]:
import numpy as np

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Pokemon.csv')

In [3]:
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [5]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [6]:
display(data.isnull().sum())

#             0
Name          0
Type 1        0
Type 2        0
Total         0
HP            0
Attack        0
Defense       0
Sp. Atk       0
Sp. Def       0
Speed         0
Generation    0
Legendary     0
dtype: int64

In [7]:
data.drop(columns=['#'], inplace=True)

In [17]:
data.drop(columns=['Name'], inplace=True)

In [18]:
data

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Grass,Poison,525,80,82,83,100,100,80,1,False
3,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Fire,No 2nd type,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [21]:
X = data.drop(columns='Legendary')
y = data['Legendary'].astype('int')

In [11]:
y.value_counts()

0    735
1     65
Name: Legendary, dtype: int64

In [12]:
y.value_counts(normalize=True)

0    0.91875
1    0.08125
Name: Legendary, dtype: float64

# Make some default pipeline

In [13]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [14]:
data.head(2)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


In [19]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [22]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

In [23]:
cv_res1

{'fit_time': array([0.19907188, 0.16871667, 0.16913414, 0.1853261 , 0.16838288]),
 'score_time': array([0.00869107, 0.00789714, 0.00791311, 0.00830603, 0.00830913]),
 'test_score': array([0.5       , 0.72727273, 0.47619048, 0.38095238, 0.64864865]),
 'train_score': array([0.71287129, 0.56097561, 0.6744186 , 0.7311828 , 0.72727273])}

In [24]:
cv_res1['test_score'].mean()

0.5466128466128467

# Make pipeline more complicated

In [25]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [26]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

cv_res2

{'fit_time': array([0.08517003, 0.13080215, 0.12634587, 0.07715607, 0.17890787]),
 'score_time': array([0.02143884, 0.01647091, 0.01552606, 0.0092988 , 0.01028419]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [27]:
cv_res2['test_score'].mean()

0.5934281842818427

train_score - просто класс ! модель получилась сложная, только очевидно переобученная ...

согласны, узнали ?


# Introduce feature selectors

In [28]:
data_tr = pipe_dif[:-1]

In [29]:
data_tr

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler())])

In [31]:
X.shape

(800, 10)

In [30]:
X_tr = data_tr.fit_transform(X, y)
print(f'data shape after transformation is {X_tr.shape}')

data shape after transformation is (800, 1001)


1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

В том числе на лекции обсуждались:

 - статистика хи-квадрат
 - метрика mutual information

In [32]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [33]:
k_best = 30

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func=mutual_info_classif, k=30)), 
    ('model_', SVC(kernel='linear'))]
)

In [34]:
cv_res = cross_validate(pipe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res

{'fit_time': array([2.67352605, 2.38835406, 2.37538624, 2.36534214, 2.37673998]),
 'score_time': array([0.00847888, 0.00846887, 0.00778699, 0.00764799, 0.00744987]),
 'test_score': array([0.26666667, 0.8       , 0.61538462, 0.42105263, 0.4       ]),
 'train_score': array([0.7755102 , 0.64516129, 0.7628866 , 0.74226804, 0.77894737])}

In [35]:
# k best нужно подбирать

cv_res['test_score'].mean()

0.5006207827260459

## Жадный метод отбора

In [36]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [37]:
rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [38]:
X_tr.shape

(800, 1001)

In [39]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

(800, 30)

array([[-1.39963712, -0.72497315, -0.80122124, ..., -0.40535388,
        -0.52135831, -0.72668962],
       [-1.39963712, -0.39095874, -0.35246512, ..., -0.16466941,
        -0.18003271, -0.70651232],
       [-1.39963712,  0.38499825,  0.66762108, ...,  0.45023305,
         0.73602435, -0.6731154 ],
       ...,
       [ 1.61195431,  1.82259714,  0.55477968, ...,  3.97831918,
         2.1076934 ,  4.87819432],
       [ 1.61195431,  3.67588727,  0.55477968, ...,  6.06069716,
         2.51608978,  4.87819432],
       [ 1.61195431,  1.44154683,  1.23182804, ...,  1.4680924 ,
         0.19750623,  1.93926564]])

In [43]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter=1000),
                      n_features_to_select=30,
                      step=30
                     )), 
    ('model_', SVC(kernel='linear'))])

In [44]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

{'fit_time': array([2.03126383, 2.26443291, 1.78478503, 2.26449084, 2.06749415]),
 'score_time': array([0.0079608 , 0.007478  , 0.00778913, 0.00773001, 0.00754905]),
 'test_score': array([0.375     , 0.84615385, 0.70967742, 0.66666667, 0.65      ]),
 'train_score': array([0.90384615, 0.84536082, 0.93457944, 0.89583333, 0.91089109])}

In [45]:
cv_res3['test_score'].mean()

0.6494995864350703

## С помощью L1 регуляризации

In [46]:
from sklearn.feature_selection import SelectFromModel

In [47]:
sel = SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=1e-5)

In [48]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

(800, 48)

array([[-1.20562657, -0.94218651, -0.7732015 , ..., -0.52135831,
        -0.72668962, -0.81966779],
       [-1.20562657, -0.94218651, -0.33767384, ..., -0.18003271,
        -0.70651232, -0.81698899],
       [-1.20562657, -0.94218651,  0.6523422 , ...,  0.73602435,
        -0.6731154 , -0.81341726],
       ...,
       [-1.20562657, -0.94218651,  0.80566326, ...,  2.1076934 ,
         4.87819432,  4.18343684],
       [-1.20562657, -0.94218651,  1.06849938, ...,  2.51608978,
         4.87819432,  4.18343684],
       [-1.20562657, -0.94218651,  2.2512619 , ...,  0.19750623,
         1.93926564,  2.6404483 ]])

In [50]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel='linear'))])

In [51]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res4

{'fit_time': array([0.11429286, 0.10220003, 0.09709072, 0.08966017, 0.098104  ]),
 'score_time': array([0.00952101, 0.00777698, 0.00816107, 0.00758004, 0.00750804]),
 'test_score': array([0.375     , 0.88      , 0.64285714, 0.60869565, 0.68421053]),
 'train_score': array([0.92307692, 0.875     , 0.95145631, 0.93877551, 0.92929293])}

In [52]:
cv_res4['test_score'].mean()

0.638152664269369

# Нелинейные классификаторы

- SVM с ядром
- Наивный байесовский классификатор
- Метод k ближайших соседей

In [53]:
pipe_lasso2 =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel='rbf'))])

In [54]:
cv_res5 = cross_validate(pipe_lasso2, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([0.10160804, 0.10773587, 0.09844184, 0.08809304, 0.09582686]),
 'score_time': array([0.01076508, 0.01063299, 0.01149225, 0.01038313, 0.01016402]),
 'test_score': array([0.35294118, 0.7       , 0.5       , 0.42105263, 0.66666667]),
 'train_score': array([0.90909091, 0.81818182, 0.83870968, 0.91666667, 0.91836735])}

In [55]:
cv_res5['test_score'].mean()

0.5281320949432404

Обучите наивный байесовский классификатор и метод k ближайших соседей вместо SVM в пайплайне выше. 

In [57]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [95]:
# your code here
pipe_knn =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', KNeighborsClassifier())])

In [96]:
pipe_knn

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler()),
                ('selector_',
                 SelectFromModel(estimator=LogisticRegression(max_iter=1000,
                                                              penalty='l1',
                                                              solver='liblinear'),
                                 threshold=1e-05)),
                ('model_', KNeighborsClassifier())])

In [97]:
cv_res5 = cross_validate(pipe_knn, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([0.10558009, 0.12564087, 0.11749411, 0.11167383, 0.12166595]),
 'score_time': array([0.0175159 , 0.01719117, 0.01791096, 0.01770115, 0.01851797]),
 'test_score': array([0.47058824, 0.72727273, 0.38095238, 0.33333333, 0.66666667]),
 'train_score': array([0.77419355, 0.79166667, 0.73333333, 0.77777778, 0.82      ])}

In [98]:
cv_res5['test_score'].mean()

0.5157626687038451

С помощью GridSearch подберите гиперпараметр KNN (число соседей) внутри пайплайна.

In [70]:
# your code here
from sklearn.model_selection import GridSearchCV

In [102]:
pipe_knn =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('gridsearch_', GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors' : range(1,51)}))])

In [103]:
cv_res5 = cross_validate(pipe_knn, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

{'fit_time': array([1.86652517, 1.91584492, 1.90953088, 1.83931422, 1.89769602]),
 'score_time': array([0.02057195, 0.01816392, 0.01721096, 0.018785  , 0.01922393]),
 'test_score': array([0.26666667, 0.7       , 0.21052632, 0.31578947, 0.48      ]),
 'train_score': array([0.47222222, 0.45714286, 0.6       , 0.54054054, 0.65060241])}

In [104]:
cv_res5['test_score'].mean()

0.39459649122807017

# Сохранение и загрузка модели

In [105]:
import pickle #консервы

Обучим лучшую модель на всех данных и сохраним её в файл.

In [ ]:
#your code here

model = ...

Сохранение модели в файл

In [ ]:
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))

Загрузка модели из файла

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

Второй вариант

In [2]:
import joblib

filename = 'best_model.sav'
joblib.dump(model, filename)
 
loaded_model = joblib.load(filename)

Почитать подробнее про сохранение модели в файл и загрузку из файла тут: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/